# Final Project Big Data
- Calssification of ratings , using Random Forest
- **Without pre-processing**

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import review_preprocess
import summarize_review

print("Setup Done..")

Setup Done..


## Loading our data_set
- please change path according to your directory

In [4]:
train_file_name = 'drugsComTrain_raw.csv'
test_file_name = 'drugsComTest_raw.csv'
file_path = '../../data/'

df_test = pd.read_csv(file_path+test_file_name)
df_train = pd.read_csv(file_path+train_file_name)

## Our data set Description
- the dataset was generated using web crawlers
- the dataset consists of DrugName, condition and a review column along with rating
- For example : if condition of patient is Obesity , then the drug prescribed to him is Contrave

In [5]:
#Print one example row of dataset
df_train[df_train.condition == 'Obesity'].iloc[1]

uniqueID                                                   39795
drugName                                                Contrave
condition                                                Obesity
review         "I am just finishing my second week taking Con...
rating                                                         8
date                                                    4-Nov-15
usefulCount                                                    9
Name: 41, dtype: object

In [6]:
df_train.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37


## PreProcess DataSet:

- What Pre-process does?
        - It removes rows with NA values (we decided to remove since
            the number of such rows is ver less)
        - It replaces garbage characters (( &#039; ,!,?,  &rsquo; , \r \n )) 
            with appropriate values
        - It then returns cleaned dataframe

In [7]:
# Print a review before cleaning
df_test.review[8]

'"Ive been on Methadone for over ten years and currently,I am trying to get off of this drug. Ive been decreasing my does 2 mgs per month for over a year. I am at 3 mgs and really starting to feel the withdraw.I don&#039;t plan to get my next 30 doses.because its almost rediculous how little it does for me. I have 3 does doses of 3 mg and Im terrified. Can anyone give me some truthful encouragement?....."'

In [8]:
df_test = review_preprocess.clean_reviews(df_test)
df_train = review_preprocess.clean_reviews(df_train)

Review cleanup Completed...aracters from review column--> 100.0 percentage complete
Removing row with nan values
Percentage of nan rows in dataset--> 0.55 %
Removed 295 rows with na values
Review cleanup Completed...aracters from review column--> 100.0 percentage complete
Removing row with nan values
Percentage of nan rows in dataset--> 0.56 %
Removed 899 rows with na values


In [9]:
# Print a review after cleaning
df_test.review[8]

"'Ive been on Methadone for over ten years and currently I am trying to get off of this drug. Ive been decreasing my does 2 mgs per month for over a year. I am at 3 mgs and really starting to feel the withdraw.I don't plan to get my next 30 doses.because its almost rediculous how little it does for me. I have 3 does doses of 3 mg and Im terrified. Can anyone give me some truthful encouragement  .'"

# Summarize our review.
- After Cleaning our review , we try to summarize it

In [10]:
%%time
import multiprocessing as mp

print("Number of processors using: ", mp.cpu_count()-1)
pool = mp.Pool(mp.cpu_count()-1)
rows,_ = df_train.shape


print("starting summarization of training set")
summary_v=np.vectorize(summarize_review.summarize)
short_review = pool.map(summary_v, df_train.review.values)
pool.close()
print("Summarization of training set COMPLETE....")

print("starting summarization of test set")
pool = mp.Pool(mp.cpu_count())
short_review_test = pool.map(summary_v, df_test.review.values)
pool.close()
print("Summarization of test set COMPLETE....")


df_train["summarize"] = np.array(short_review)
df_test["summarize"] = np.array(short_review_test)


Number of processors using:  7
starting summarization of training set
Summarization of training set COMPLETE....
starting summarization of test set
Summarization of test set COMPLETE....
Wall time: 7min 31s


In [34]:
print('-------original review-------')
print(df_test.review[10])
print('\n')
print('-------summarized review-------')
print(df_test.summarize[10])

-------original review-------
'Holy Hell is exactly how I feel. I had been taking Brisdelle for 1.5 years. The hot flashes did indeed subside - however  the side affects of this medicine coupled with the fact Noven was acquired by YET another pharmaceutical company - YOU CAN'T PLACE A REP IN THE AREA  DISTRIBUTE YOUR DRUGS  AND THEN FIRE HER-AND NOT REPLACE THEREFORE there is NO medicine or support here. You dumped this drug in the Dr's hands and walked away. After calling Sebula - you act like you don't even care. You have made it impossible to obtain this. I happen to think this is illegal.  I just decided to wean myself off this and Premarin. It has been nothing short of a nightmare. If you don't need this drug- DON'T START. Seriously.'


-------summarized review-------
 You have made it impossible to obtain this. I happen to think this is illegal. I just decided to wean myself off this and Premarin. It has been nothing short of a nightmare. If you don't need this drug- DON'T START.

# Vectorizing our review column

### - why vectorize?
       1). Our Review column have different reviews of different length 
       to convet each review into same size we use vectorizing.
       
       2). Vectorizing involves Tokenizing and then padding each sentence.
       We have aprox 49K unique words in our review columns so,
       the number of features should be around 49K after vectorizing.
       
### - TF-IDF vectorizer :
- we have used Tf-IDF vectorizer . More Details about TF-IDF is present in Project report
- For the time being we can assume it is used to convert each review to equal number of features.

In [13]:
# Creates TF-IDF vectorizer and transforms the corpus
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(df_train.review)

# transforms test reviews to above vectorized format
X_test = vectorizer.transform(df_test.review)
print("Vectorizing done")

Vectorizing done


In [32]:
# Creates TF-IDF vectorizer and transforms the corpus
vectorizer_sum = TfidfVectorizer()
X_train_summary = vectorizer_sum.fit_transform(df_train.summarize)

# transforms test reviews to above vectorized format
X_test_summary = vectorizer_sum.transform(df_test.summarize)
print("Vectorizing done")

Vectorizing done


In [11]:
print("Test Shape : ",X_test.shape)
print("Train Shape :",X_train.shape)
print("We can see that every review in\
test and train Dataset have equal number of features now")

Test Shape :  (53471, 49853)
Train Shape : (160398, 49853)
We can see that every review intest and train Dataset have equal number of features now


In [12]:
# Assign labels y_train and y_test
y_train = df_train.rating.copy(deep=True)
y_test = df_test.rating.copy(deep=True)


## Random Forest Classifier
- we have performed a grid seach of number of trees and m (number of features per tree)

In [ ]:
%%time
#
_,features = X_test.shape
accuracy_list = {}


## our grid exaustive search parameters
m_features_names = ["2 sqrt(M)", "sqrt(M)", "log2(M)"]
m_features = [int(2*math.sqrt(features)), int(math.sqrt(features)), int(math.log2(features))]
num_trees = [10,20,40,50,100,150]

for m,m_name in zip(m_features,m_features_names):
    accuracy_list[m_name] = []
    for tr in num_trees:
        rfc_rating = RandomForestClassifier(n_estimators=tr,max_features=m,n_jobs=-1,verbose=2, random_state=42 )
        rfc_rating.fit(X_train, y_train)
        accuracy = rfc_rating.score(X_test, y_test)
        accuracy_list[m_name].append(accuracy)



building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:  1.1min remaining:   29.4s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.5min finished
[Parallel(n_jobs=8)]: Done   7 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.1s finished


building tree 1 of 20building tree 2 of 20
building tree 3 of 20
building tree 4 of 20
building tree 5 of 20
building tree 6 of 20
building tree 7 of 20
building tree 8 of 20

building tree 9 of 20
building tree 10 of 20
building tree 11 of 20
building tree 12 of 20
building tree 13 of 20
building tree 14 of 20
building tree 15 of 20
building tree 16 of 20
building tree 17 of 20
building tree 18 of 20
building tree 19 of 20
building tree 20 of 20


[Parallel(n_jobs=-1)]: Done  16 out of  20 | elapsed:  1.9min remaining:   28.2s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  2.4min finished
[Parallel(n_jobs=8)]: Done  16 out of  20 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  20 out of  20 | elapsed:    0.2s finished


building tree 1 of 40
building tree 2 of 40
building tree 3 of 40
building tree 4 of 40
building tree 5 of 40
building tree 6 of 40
building tree 7 of 40
building tree 8 of 40
building tree 9 of 40
building tree 10 of 40
building tree 11 of 40
building tree 12 of 40
building tree 13 of 40
building tree 14 of 40
building tree 15 of 40
building tree 16 of 40
building tree 17 of 40
building tree 18 of 40
building tree 19 of 40
building tree 20 of 40
building tree 21 of 40
building tree 22 of 40
building tree 23 of 40
building tree 24 of 40
building tree 25 of 40
building tree 26 of 40
building tree 27 of 40
building tree 28 of 40
building tree 29 of 40
building tree 30 of 40
building tree 31 of 40
building tree 32 of 40


...

building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 50 of 50


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  5.6min finished
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.6s finished


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  3.3min


building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100


In [42]:
X_test.shape

(53471, 34136)

In [40]:
pred = rfc_rating.predict(X_test)

print("Accuracy: %s" % str(rfc_rating.score(X_test, y_test)))

[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.6s finished
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.3s


Accuracy: 0.7352396626208599


[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.6s finished


In [28]:
from sklearn.metrics import classification_report

target_names = ['class 0', 'class 1', 'class 2','class 3',\
                'class 4', 'class 5','class 6', 'class 7', 'class 8','class 9']
print(classification_report(y_test, pred, target_names=target_names))

             precision    recall  f1-score   support

    class 0       0.76      0.77      0.76      7299
    class 1       0.97      0.63      0.76      2334
    class 2       0.97      0.63      0.76      2205
    class 3       0.98      0.63      0.76      1659
    class 4       0.98      0.60      0.75      2710
    class 5       0.98      0.59      0.73      2119
    class 6       0.97      0.58      0.73      3091
    class 7       0.90      0.61      0.73      6156
    class 8       0.81      0.62      0.70      9177
    class 9       0.61      0.95      0.75     17016

avg / total       0.79      0.74      0.74     53766

